I. Environment setup: 
- Spark 3.1.2 with Hadoop 3.2.
- Java 8.

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz
!tar xf spark-3.1.2-bin-hadoop3.2.tgz
!pip install -q findspark

In [3]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop3.2"

II. Code: 

In [5]:
pyspark

NameError: ignored

In [ ]:
textFile = spark.read.text("README.md")

In [ ]:
textFile.count()